In [1]:
import pandas as pd
from urllib.parse import quote
from google.colab import files
import numpy as np

In [3]:
df= pd.read_csv('/content/miniseries.csv',encoding='utf-8')
df=df.loc[df['방송'].str.len()>1]
df

,Unnamed: 0,작품명,극본,연출,방송 기간,방송,방송사,유형,UHD,방송.1,시청,비고
0,3,국민 여러분!,한정훈,김정현,"2019.05.28, 2019.04.01",36 (18),KBS2,월화,NaN,NaN,(),8회 연장
1,10,어쩌다 발견한 하루,"인지혜, 송하영",김상협,"2019.10.02, 2019.11.21",32 (16),MBC,수목,NaN,NaN,(),사전제작
2,12,위대한 쇼,설준석,신용휘,"2019.10.15, 2019.8.26",16,tvN,월화,NaN,NaN,(),21회 조기종영
3,13,유령을 잡아라,"이영주, 소원",신윤섭,"2019.10.21, 2019.12.10",16,tvN,월화,NaN,NaN,(),사전제작
4,16,달리는 조사관,백정철,"김용수, 이창희","2019.10.31, 2019.9.18",14,OCN,수목,NaN,NaN,(),2회 조기종영
...,...,...,...,...,...,...,...,...,...,...,...,...
200,411,"어쩌다 마주친, 그대",백소연,"강수연, 이웅희","2023.6.20, 2023.5.1",16,KBS2,월화,NaN,NaN,(),사전제작
201,415,가슴이 뛴다,"정승주, 김하나","이민수, 이현석",2023.6.26,16,KBS2,월화,NaN,NaN,(),사전제작
202,416,기적의 형제,김지우,박찬홍,2023.6.28,16,JTBC,수목,NaN,NaN,(),사전제작
203,418,이로운 사기,한우주,이수현,"2023.7.18, 2023.5.29",16,tvN,월화,NaN,NaN,(),사전제작


In [ ]:
'''url = "https://ko.wikipedia.org/wiki/동백꽃 필 무렵"
encoded_url = quote(url, safe=':/_')'''

In [4]:
drama_list = df['작품명'].tolist()

In [12]:
meta1=pd.DataFrame()

for name in drama_list:
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)
    table = tables[0]

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

        # 이전 이후 드라마
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          break
      except Exception:
        pass
    df3.columns = ['이전', '현재', '다음']
    table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table=tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

            # 이전 이후 드라마
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            break
        except Exception:
          pass
      df3.columns = ['이전', '현재', '다음']
      table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

    except Exception:
      table = title

  meta1=pd.concat([meta,table],ignore_index=True)


In [13]:
meta1

,작품명,장르,방송 채널,방송 기간,방송 시간,방송 분량,방송 횟수,기획,책임프로듀서,프로듀서,제작사,연출,각본,출연자,이전,다음
0,국민 여러분!,"범죄, 코미디",KBS 2TV,2019년 4월 1일 ~ 2019년 5월 28일,"매주 월요일, 화요일 밤 10시 ~ 11시 10분",1시간 10분 (1회당 35분 / 2회 연속방송),36부작,문준하 (KBS 드라마본부),"이건준, 노상훈 (KBS 드라마운영팀)",NaN,"몬스터유니온, 원콘텐츠",김정현,한정훈,"최시원, 이유영, 김민정, 태인호, 김의성 외",동네변호사 조들호 2: 죄와 벌 (2019년 1월 7일 ~ 2019년 3월 26일),퍼퓸 (2019년 6월 3일 ~ 2019년 7월 23일)
1,어쩌다 발견한 하루,"로맨틱 판타지, 로맨스 드라마",MBC,2019년 10월 2일 ~ 2019년 11월 21일,"매주 수, 목 저녁 8:55 ~ 밤 10:05",1시간 10분 (1회당 35분),32부작,김대진,NaN,"남궁성우, 문주희",MBC 드라마본부 래몽래인[1],김상협,"인지혜, 송하영","김혜윤, 로운, 이재욱, 이나은, 정건주, 김영대, 이태리 외",신입사관 구해령 (2019년 7월 17일 ~ 2019년 9월 26일),하자있는 인간들 (2019년 11월 27일 ~ 2020년 1월 16일)
2,위대한 쇼,드라마,tvN,2019년 8월 26일 ~ 2019년 10월 15일,"매주 월, 화 밤 9시 30분",NaN,16부작,NaN,NaN,NaN,"화이브라더스코리아, 롯데컬처웍스","신용휘, 김정욱",설준석,"송승헌, 이선빈, 임주환 외","60일, 지정생존자 (2019년 7월 1일 ~ 2019년 8월 20일)",유령을 잡아라! (2019년 10월 21일 ~ 2019년 12월 10일)
3,유령을 잡아라,드라마,tvN,2019년 10월 21일 ~ 2019년 12월 10일,"매주 월, 화 밤 9시 30분",NaN,16부작,스튜디오드래곤,NaN,NaN,로고스 필름 (주),신윤섭,"소원, 이영주","문근영, 김선호 외",위대한 쇼 (2019년 8월 26일 ~ 2019년 10월 15일),블랙독 (2019년 12월 16일 ~ 2020년 2월 4일)
4,달리는 조사관,휴먼,OCN,2019년 9월 18일 ~ 2019년 10월 31일,"매주 수요일, 목요일 밤 11시",60분,14부작,스튜디오드래곤,NaN,NaN,데이드림엔터테인먼트,김용수,백정철,"이요원, 최귀화 외",미스터 기간제 (2019년 7월 17일 ~ 2019년 9월 5일),-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,가슴이 뛴다,"공포, 로맨스",KBS 2TV,2023년 6월 26일 ~ 2023년 8월 15일,"매주 월요일, 화요일 밤 9시 45분 ~ 10시 55분",NaN,16부작,박기호 (KBS 드라마본부),기민수 (KBS 드라마운영팀),우예스란,"위매드, 몬스터유니온","이현석, 이민수",NaN,"옥택연, 원지안, 박강현, 윤소희 外","어쩌다 마주친, 그대 (2023년 5월 1일 ~ 2023년 6월 20일)",순정복서 (2023년 8월 21일 ~ 방송 예정)
200,기적의 형제,드라마,JTBC,2023년 6월 28일~2023년 8월 17일,"매주 수요일, 목요일 밤 10:30~",70분,16부작,SLL,총괄프로듀서: 박슬기,"전서원, 김유리, 김수지, 조동민","MI, SLL",NaN,NaN,"정우, 배현성, 박유림, 오만석, 이기우 외",나쁜 엄마 (2023년 4월 26일~2023년 6월 8일),이 연애는 불가항력 (2023년 8월 23일~)
201,이로운 사기,"케이퍼, 범죄, 복수, 피카레스크, 드라마",tvN,2023년 5월 29일 ~ 2023년 7월 18일,"매주 월요일, 화요일 밤 8시 50분 ~ 10시 10분",1시간 20분,16부작,스튜디오드래곤,"소재현, 박은경","김겨레, 이민아",스튜디오드래곤,이수현,NaN,NaN,패밀리 (2023년 4월 17일 ~ 2023년 5월 23일),소용없어 거짓말 (2023년 7월 31일 ~ )
202,행복배틀,"서스펜스, 스릴러",ENA,2023년 5월 31일 ~ 2023년 7월 20일,"매주 수요일, 목요일 밤 9:00 ~ 10:25",70분,16부작,KT스튜디오지니,"김현정, 제작총괄: 정태중","김현철, 이희원",HB엔터테인먼트,김윤철,NaN,"이엘, 진서연, 차예련, 박효주, 우정원 외",보라! 데보라 (2023년 4월 12일 ~ 2023년 5월 25일),오랫동안 당신을 기다렸습니다 (2023년 7월 26일 ~ 2023년 9월 7일)


In [14]:
meta1[meta1['장르'].isna()]["작품명"].unique()

array(['보좌관 2', 'VIP', '아이템', '해치', '빙의', '빅이슈', '어비스', '바람이 분다', '퍼퓸',
       '레벨업', '에이틴 / 에이틴 시즌 2', '저스티스', '나를 사랑한 스파이', '카이로스', '나의 위험한 아내',
       nan, '사생활', '낮과 밤', '블랙독', '방법', '포레스트', '머니게임', '아무도 모른다',
       '본 어게인', '저녁 같이 드실래요?', '오 마이 베이비', '출사표', '악의 꽃', '마우스', '나빌레라',
       '로스쿨', '멀리서 보는 푸른 봄', '홈타운', '펜트하우스', '연모', '스폰서', '크레이지 러브',
       '인사이더', '이브', '치얼 업', '커튼 콜', '트롤리', '오아시스', '나쁜 엄마', '패밀리'],
      dtype=object)

In [15]:
error_list = ['보좌관_2_-_세상을_움직이는_사람들','VIP_(드라마)','아이템_(드라마)','해치_(드라마)','빙의_(드라마)','빅이슈_(2019년_드라마)','어비스_(드라마)','바람이_분다_(드라마)','퍼퓸_(드라마)','레벨업_(드라마)','에이틴_(드라마)','에이틴_2','저스티스_(드라마)','나를_사랑한_스파이_(드라마)','카이로스_(드라마)','나의_위험한_아내_(대한민국의_드라마)','사생활_(드라마)','낮과_밤_(드라마)','블랙독_(드라마)','방법_(드라마)','포레스트_(드라마)','머니게임_(드라마)','아무도_모른다_(드라마)','본_어게인_(드라마)','저녁_같이_드실래요%3F_(드라마)','오_마이_베이비_(드라마)','출사표_(드라마)','악의_꽃_(2020년_드라마)','마우스_(드라마)','나빌레라_(드라마)','로스쿨_(드라마)','멀리서_보면_푸른_봄_(드라마)','홈타운_(드라마)','펜트하우스_(드라마)','연모_(드라마)','스폰서_(드라마)','크레이지_러브_(드라마)','인사이더_(드라마)','이브_(2022년_드라마)','치얼업_(드라마)','커튼콜_(드라마)','트롤리_(드라마)','오아시스_(드라마)','나쁜_엄마_(드라마)','패밀리_(드라마)']

In [16]:
meta2=pd.DataFrame()

for name in error_list:
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)
    table = tables[0]

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

        # 이전 이후 드라마
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          break
      except Exception:
        pass
    df3.columns = ['이전', '현재', '다음']
    table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table=tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

            # 이전 이후 드라마
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            break
        except Exception:
          pass
      df3.columns = ['이전', '현재', '다음']
      table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

    except Exception:
      table = title

  meta2=pd.concat([meta,table],ignore_index=True)


<ipython-input-16-656b3f515e8c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[col] = np.nan
<ipython-input-16-656b3f515e8c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[col] = np.nan
<ipython-input-16-656b3f515e8c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [17]:
meta = pd.concat([meta1, meta2], ignore_index = True)

In [20]:
meta.to_csv('미니시리즈_메타데이터.csv', index=False, encoding='utf-8-sig')
files.download('미니시리즈_메타데이터.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 시청률 데이터

In [ ]:
# 시청률
for table in tables:
  if table.iloc[0,0] == '제1회':
    df2 = table.iloc[:, :-1]
    break

df2.columns = [tpl[-1] for tpl in df2.columns[:]]                 # 멀티인덱스 정리
df2 = df2[['회차', '방송일', '대한민국(전국)', '서울(수도권)']]   # 필요한 열만 호출

df2 = df2.loc[:df2[df2['회차'] == '평균 시청률'].index[0]-1]      # 평균 시청률 행 위까지가 회차별 데이터

# 데이터 확인 & 다운로드

In [ ]:
df1

,장르,방송 채널,방송 기간,방송 시간,방송 분량,방송 횟수,기획,책임프로듀서,프로듀서,제작사,연출,각본,출연자,이전,다음
0,"멜로, 휴머니즘, 스릴러",KBS 2TV,2019년 9월 18일 ~ 2019년 11월 21일,"매주 수요일, 목요일 밤 10:00 ~ 11:10",1시간 10분 (1회당 35분),40부작 + 스페셜 4부작,황의경 (KBS 드라마본부),"이건준, 노상훈 (KBS 드라마운영팀)",윤재혁,팬 엔터테인먼트,"차영훈, 강민경",임상춘,"공효진, 강하늘, 김지석, 지이수, 손담비 외",저스티스 (2019년 7월 17일 ~ 2019년 9월 5일),99억의 여자 (2019년 12월 4일 ~ 2020년 1월 23일)


In [ ]:
df2

,회차,방송일,대한민국(전국),대한민국(전국),서울(수도권)
0,제1회,9월 18일,7.4%,6.3%,6.7%
1,제2회,9월 18일,8.7%,7.4%,7.6%
2,제3회,9월 19일,6.4%,6.7%,7.0%
3,제4회,9월 19일,7.9%,8.3%,8.7%
4,제5회,9월 25일,7.5%,8.6%,9.4%
5,제6회,9월 25일,8.7%,10.0%,11.0%
6,제7회,9월 26일,7.5%,7.7%,7.4%
7,제8회,9월 26일,9.2%,10.0%,10.0%
8,제9회,10월 2일,8.4%,9.3%,10.1%
9,제10회,10월 2일,9.3%,11.5%,12.4%


In [ ]:
# df1.to_csv('동백꽃필무렵_메타(예시).csv', index=False, encoding='utf-8-sig')
# files.download('동백꽃필무렵_메타(예시).csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# df2.to_csv('동백꽃필무렵_시청률(예시).csv', index=False, encoding='utf-8-sig')
# files.download('동백꽃필무렵_시청률(예시).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>